In [2]:
import time
from pprint import pprint

from tqdm import tqdm_notebook as tqdm
import re

import numpy as np

import sklearn
import sklearn.ensemble, sklearn.metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.datasets import fetch_20newsgroups
from sklearn.pipeline import make_pipeline
from sklearn import linear_model

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Helper functions

In [3]:
def sanitize_text(text):
    """
    Sanitizes text by
    - converting all text to lowercase
    - removing newlines and replacing by space
    - replacing everything that is not a-z or apostrophe or space to a space
    """
    sanitized_text = text.lower()
    
    sanitized_text = re.sub(r"\n"," ", sanitized_text)
    sanitized_text = re.sub(r"[^a-z']+"," ", sanitized_text)
    
    return sanitized_text

## LIME implementation

In [4]:
def similarity_kernel(x, z, sigma=1, distance=cosine_similarity):
    """ The data-point distance kernel as described in the paper """
    return np.exp( -np.power(distance(x,z),2)/np.power(sigma,2) )

In [5]:
def sample_around(x_text_vec):
    """ 
    Method for sampling an instance that is in the vicinity
    of x_text_vec as described by the paper 
    """
    x_text_vec = np.array(x_text_vec, copy=True)  

    max_samples = x_text_vec.shape[1]
    num_samples = np.random.randint(max_samples, size=1)
    indices = np.random.choice(range(max_samples), num_samples, replace=False)

    x_text_vec[:,~indices] = 0
    return x_text_vec

In [6]:
def feature_selection_lasso_path(features, labels, weights, num_features=10):
    """
    Feature selection using lasso path, taken from the authors 
    github(https://github.com/marcotcr/lime) page and slightly modified.
    """
    labels = labels[:,0] #we use the first probability
    weighted_data = np.multiply(features - np.average(features, axis=0, weights=weights), 
                                  np.sqrt(weights[:, np.newaxis])) #weights[:, np.newaxis]=(d,)->(d,1) conversion
    weighted_labels = np.multiply(labels - np.average(labels, weights=weights), 
                                  np.sqrt(weights))
    
    _,_,coefs = linear_model.lars_path(weighted_data,
                                 weighted_labels,
                                 method='lasso',
                                 verbose=False)

    nonzero = range(weighted_data.shape[1])
    for i in range(len(coefs.T) - 1, 0, -1):
        nonzero = coefs.T[i].nonzero()[0]
        if len(nonzero) <= num_features:
            break
    used_features = nonzero
    return used_features

In [7]:
def sparse_linear_expl_LIME(x_instance, num_samples, clf, sim_kernel, expl_length):
    """
    Trains a linear explanator around a vicinity of a data point, in this case, a text
    x_instance: a text to be classified and then explained.
    
    """
    sim_kernel_x = lambda z: sim_kernel(x_text_vec, z)
    
    mlb = MultiLabelBinarizer()
    x_text_vec = mlb.fit_transform([x_instance.split(" ")])
    
    Z_features = []
    Z_labels = []
    Z_sim_kernel = []
    for i in tqdm(range(num_samples)):
        z_text_vec = sample_around(x_text_vec)
        z_text = " ".join(mlb.inverse_transform(z_text_vec)[0])
        Z_features.append( z_text_vec )

        Z_labels.append( clf.predict_proba([z_text])[0][1] )
        Z_sim_kernel.append( sim_kernel_x(z_text_vec) )
        
    local_train_X = np.vstack(Z_features)
    local_train_y = np.vstack(Z_labels)
    weights = np.hstack(Z_sim_kernel)[0]

    selected_features = feature_selection_lasso_path(local_train_X, local_train_y, weights, expl_length)

    explanator_clf = linear_model.Ridge(alpha=1, fit_intercept=True)
    explanator_clf.fit(local_train_X[:, selected_features], local_train_y)
    
    return explanator_clf.coef_, mlb.classes_[selected_features]

## Split data and train black box model

In [7]:
newsgroups_train = fetch_20newsgroups(subset='train', shuffle=False)
newsgroups_test = fetch_20newsgroups(subset='test', shuffle=False)
pprint(list(newsgroups_train.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [60]:
selected_cat = ['alt.atheism', 'soc.religion.christian']

# Used neat Sklearn method for both fetching and splitting the dataset
newsgroups_train = fetch_20newsgroups(subset='train', categories=selected_cat, shuffle=True)
newsgroups_test = fetch_20newsgroups(subset='test', categories=selected_cat, shuffle=True)

In [64]:
instance_index = 15 # "Randomly" selected instance
x_instance_unsanitized = newsgroups_test.data[instance_index]
x_instance_sanitized = sanitize_text(x_instance_unsanitized)

x_instance_target = newsgroups_test.target[instance_index]

for enum, text in enumerate(newsgroups_train.data):
    newsgroups_train.data[enum] = sanitize_text(text)
    
for enum, text in enumerate(newsgroups_test.data):
    newsgroups_test.data[enum] = sanitize_text(text)

### Preprocess data by feature extraction using TF-FID transformation

In [65]:
# Preprocessing text data for black box supervised training.
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = vectorizer.transform(newsgroups_test.data)

### Train black box on transformed text data

In [66]:
black_box = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
black_box.fit(train_vectors, newsgroups_train.target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [67]:
pred = black_box.predict(test_vectors)
sklearn.metrics.roc_auc_score(newsgroups_test.target, pred)

0.9177288528389339

In [68]:
model_pipeline = make_pipeline(vectorizer, black_box)

## Test LIME implementation

In [69]:
coefs, selected_words = sparse_linear_expl_LIME(x_instance_sanitized, 5000, model_pipeline, similarity_kernel, 6)

In [70]:
print(x_instance_unsanitized)
print("\n\nMost explanatory features: {}".format(selected_words[coefs[0].argsort()][::-1]))
print("Most explanatory feature coeffs: {}".format(coefs[0][coefs[0].argsort()][::-1]))

From: nyeda@cnsvax.uwec.edu (David Nye)
Subject: College atheists
Organization: University of Wisconsin Eau Claire
Lines: 10

I read an article about a poll done of students at the Ivy League
schools in which it was reported that a third of the students
indentified themselves as atheists.  This is a lot higher than among the
general population.  I wonder what the reasons for this discrepancy are?
Is it because they are more intelligent?  Younger?  Is this the wave of
the future?
 
David Nye (nyeda@cnsvax.uwec.edu).  Midelfort Clinic, Eau Claire WI
This is patently absurd; but whoever wishes to become a philosopher
must learn not to be frightened by absurdities. -- Bertrand Russell



Most explanatory features: ['from' 'edu' 'than' 'more' 'article' 'atheists']
Most explanatory feature coeffs: [ 0.07330597 -0.02757331 -0.03154749 -0.03592764 -0.06914742 -0.07753285]


In [71]:
black_box_pred = model_pipeline.predict_proba([x_instance_sanitized])
print("Black box model prediction: {}".format(black_box_pred))
print("Correct label: {}".format(x_instance_target))

Black box model prediction: [[0.346 0.654]]
Correct label: 0


### Lets see if the explanator is correct by investigating removal of "explanatory" features!
#### If the coefficients are positive, the should decrease the probability of the positive class, if they are negative they should decrease the probability of the negative class ( zero in our case ).

In [72]:
x_instance_vec = vectorizer.transform([x_instance_sanitized]).copy()
x_instance_vec[0, vectorizer.vocabulary_['atheists']] = 0
x_instance_vec[0, vectorizer.vocabulary_['keith']] = 0
new_prob = black_box.predict_proba(x_instance_vec)
print("Black box model prediction after feature removal: {}".format(new_prob))
print("Correct label: {}".format(x_instance_target))

C:\ProgramData\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Black box model prediction after feature removal: [[0.29 0.71]]
Correct label: 0


#### According to author, this removal should move the prediction towards the opposite class by the sum of all weights corresponding to the removed features, we can see this behavior above, this means our explanator model successfully learned how our black box model behaves in the vicinity of the data point, in our case x_instance

# Lets see if we can apply this on a new dataset, and try to trust our new model!

In [163]:
selected_cat2 = ['rec.sport.baseball','rec.sport.hockey']
newsgroups_train2 = fetch_20newsgroups(subset='train', categories=selected_cat2, shuffle=False)
newsgroups_test2 = fetch_20newsgroups(subset='test', categories=selected_cat2, shuffle=False)

In [164]:
instance_index2 = 17 # "Randomly" selected instances for report: 17, 54, and 82
x_instance_unsanitized2 = newsgroups_test2.data[instance_index2]
x_instance_sanitized2 = sanitize_text(x_instance_unsanitized2)


x_instance_target2 = newsgroups_test2.target[instance_index2]

for enum, text in enumerate(newsgroups_train2.data):
    newsgroups_train2.data[enum] = sanitize_text(text)
    
for enum, text in enumerate(newsgroups_test2.data):
    newsgroups_test2.data[enum] = sanitize_text(text)

In [165]:
print(x_instance_unsanitized2)

From: yatrou@INRS-Telecom.Uquebec.CA (Paul Yatrou)
Subject: Re: Stewart homered the Wings!!
Organization: Bell-Northern Research Montreal, Canada.
Lines: 25

In <andy.bgsu.edu-250493225109@m248-100.bgsu.edu> andy.bgsu.edu (Ryan ) writes:

>  Paul Stewart called *the* single worst game I've seen this year.
> Federov's major was obvious, and I don't dispute it.  
>However, Chaisson's penalty shouldn't even have been a penalty, let alone
> a major and a game misconduct.
>

I don't "notice" refs and linesmen until the playoffs come around, and
yes I have to agree that Stewart called the *two* worst games I've seen
so far (Mtl-Quebec game 1, and last nights Toronto-Detroit game).

What's the scoop on this guy? Is he the latest incarnation of
KERRY FRASER??? Just because you are boneheadedly stubborn doesn't
make you a good ref!!! Making the right call does...

My votes for:
Best Ref: Van Hellemond
Most Improved: Koharski
Worst: Paul Stewart

(Oops, I don't really want to start a best/worst 

In [157]:
# Preprocessing text data for black box supervised training.
vectorizer2 = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors2 = vectorizer2.fit_transform(newsgroups_train2.data)
test_vectors2 = vectorizer2.transform(newsgroups_test2.data)

In [114]:
black_box2 = sklearn.tree.DecisionTreeClassifier()
black_box2.fit(train_vectors2, newsgroups_train2.target)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [147]:
important_feture = black_box2.feature_importances_.argsort()[-1]
vectorizer2.get_feature_names()[important_feture]

'hockey'

In [116]:
pred_train = black_box2.predict(train_vectors2)
pred_test = black_box2.predict(test_vectors2)
train_f1 = sklearn.metrics.roc_auc_score(newsgroups_train2.target, pred_train)
test_f1 = sklearn.metrics.roc_auc_score(newsgroups_test2.target, pred_test)
print("Train perf: {}, Test perf: {}".format(train_f1, test_f1))

Train perf: 1.0, Test perf: 0.8718332354816513


In [117]:
model_pipeline2 = make_pipeline(vectorizer2, black_box2)
coefs2OF, selected_words2OF = sparse_linear_expl_LIME(x_instance_sanitized2, 5000, model_pipeline2, similarity_kernel, 6)

In [118]:
print(x_instance_unsanitized2)
print("\n\nMost explanatory features: {}".format(selected_words2OF[coefs2OF[0].argsort()][::-1]))
print("Most explanatory feature coeffs: {}".format(coefs2OF[0][coefs2OF[0].argsort()][::-1]))

From: mre@teal.Eng.Sun.COM (Mike Eisler)
Subject: Re: LET'S GO BUFFALO!
Organization: Sun Microsystems, Mountain View, CA  USA
Lines: 16
NNTP-Posting-Host: teal

In article <93111.205214RAP115@psuvm.psu.edu> Robbie Po <RAP115@psuvm.psu.edu> writes:
>In article <AfpIKNm00WBLI1isJ1@andrew.cmu.edu>, "William K. Willis"
><ww1a+@andrew.cmu.edu> says:
>>
>>     You know, I never really appreciated them before!
>
>Looks like Bob Errey's ring really sparkles in that locker room, and everyone
>else wants one, too! :-)  Correct me if I'm wrong though, (just through

No, Fuhr's 5 rings out sparkle Errey's. And doesn't Bob have 2 rings?
-- 
Mike Eisler, mre@Eng.Sun.Com  ``Not only are they [Leafs] the best team, but
 their fans are even more intelligent and insightful than Pittsburgh's.  Their
 players are mighty bright, too.  I mean, he really *was* going to get his
 wallet back, right?'' Jan Brittenson 3/93, on Leaf/Pen woofers in
 rec.sport.hockey



Most explanatory features: ['hockey' 'ca' "p

In [120]:
black_box_pred = model_pipeline2.predict_proba([x_instance_sanitized2])
print("Black box model prediction: {}".format(black_box_pred))
print("Correct label: {}={}".format(x_instance_target2, selected_cat2[x_instance_target2]))

Black box model prediction: [[0. 1.]]
Correct label: 1=rec.sport.hockey


### Lets compare this to hyperparameter fitted black box model

In [148]:
"""
# Lets find some good parameters using random search bith k-fold cross validation!
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# Number of trees in random forest
n_estimators = [100]#[int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap
              }


black_box_tofit = sklearn.ensemble.RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = black_box_tofit, param_distributions = random_grid, n_iter=100, cv = 2, verbose=2)
# Fit the random search model
rf_random.fit(train_vectors2, newsgroups_train2.target)
black_box_fitted = rf_random.best_estimator_

"""

In [158]:
# The resuling model from the random search k-fold cross validation
black_box_fitted = sklearn.ensemble.RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=110, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

black_box_fitted.fit(train_vectors2, newsgroups_train2.target)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=110, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [159]:
pred_train = black_box_fitted.predict(train_vectors2)
pred_test = black_box_fitted.predict(test_vectors2)
train_f1 = sklearn.metrics.roc_auc_score(newsgroups_train2.target, pred_train)
test_f1 = sklearn.metrics.roc_auc_score(newsgroups_test2.target, pred_test)
print("Train perf: {}, Test perf: {}".format(train_f1, test_f1))

Train perf: 0.9891624790619765, Test perf: 0.9309703730358642


In [160]:
model_pipeline2 = make_pipeline(vectorizer2, black_box_fitted)
coefs2, selected_words2 = sparse_linear_expl_LIME(x_instance_sanitized2, 5000, model_pipeline2, similarity_kernel, 6)

In [161]:
print("\n\nMost explanatory features: {}".format(selected_words2[coefs2[0].argsort()][::-1]))
print("Most explanatory feature coeffs: {}".format(coefs2[0][coefs2[0].argsort()][::-1]))



Most explanatory features: ['playoffs' 'wings' 'quebec' 'toronto' 'ca' 'detroit']
Most explanatory feature coeffs: [0.08724327 0.06841966 0.05950898 0.05446195 0.04711267 0.04399823]


In [162]:
black_box_pred = model_pipeline2.predict_proba([x_instance_sanitized2])
print("Black box model prediction: {}".format(black_box_pred))
print("Correct label: {}={}".format(x_instance_target2, selected_cat2[x_instance_target2]))

Black box model prediction: [[0.25190144 0.74809856]]
Correct label: 1=rec.sport.hockey
